In [1]:
!pip install -qU langchain langchain_community langchain_chroma langchainhub pypdf langchain-ollama langchain-text-splitters

In [2]:
import os
HOME = os.getcwd()
print(HOME)

ROOT = os.path.dirname(HOME)
print(ROOT)

/Users/shubhamrathod/PycharmProjects/RAG_Pipeline/RAG_Chain
/Users/shubhamrathod/PycharmProjects/RAG_Pipeline


# Load the Documents

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = f'{HOME}/CC.pdf'
loader = PyPDFLoader(file_path = file_path)

pages = loader.load()

In [4]:
len(pages)

5

# Split the Document

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=8000,
    chunk_overlap=3000,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
docs = text_splitter.split_documents(pages)

In [8]:
len(docs)

5

# Embedding Model

In [10]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

# Create Vector DB

In [11]:
from langchain_chroma import Chroma

In [12]:
persist_directory = f'{HOME}/chroma_db'

if os.path.isdir(persist_directory):
    # Load from disk.
    index = Chroma(persist_directory = persist_directory, embedding_function = embeddings)
else:
    # Save to disk.  
    index = Chroma.from_documents(documents = docs, embedding = embeddings, persist_directory = persist_directory)

In [13]:
docs = index.similarity_search('Who is emily')
print(docs)

[Document(metadata={'page': 1, 'source': '/Users/shubhamrathod/PycharmProjects/RAG_Pipeline/RAG_Chain/CC.pdf'}, page_content='the bill that you received was generated a week before you made the \npayment that’s why your latest payment had not been reflected. You can \nsimply disregard the amount indicated  in the bill and continue enjoying \nour servic es. \nCustomer:     Thank you.  \nAgent:           Delays in the bill is usually caused by delays in our courier services. \nFor a more up dated bill of your account, you can visit our website and \nlog in to your account. This bill is more updated.  \nCustomer:     O k I will.'), Document(metadata={'page': 3, 'source': '/Users/shubhamrathod/PycharmProjects/RAG_Pipeline/RAG_Chain/CC.pdf'}, page_content='the documents from and deliver the sim to you. May I know the convenient time \nwhen you would be Available?  \n \nCustomer:  Ok, send your executive at 11 am tomorrow.  \n \nAgent:  Ok, sir. Thank you very mu ch. Have a nice day.  \n \nC

# Create Retriever

In [14]:
retriever = index.as_retriever()

# History Aware Prompt

In [25]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [30]:
llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0,
)

In [31]:
contextualize_q_system_prompt = (
    """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."""
)

In [32]:
contextualize_q_system_prompt

'Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.'

In [33]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [34]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Prompt - Chain

In [43]:
system_prompt = """

    You are an assistant for question-answering tasks. Your Name is PLN, Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    
    Context: {context} 

"""

In [44]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# Run Chain

In [57]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [52]:
chat_history = []

In [53]:
# question = "Which Bank did Tracy go to?"
# ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

# chat_history.extend(
#     [
#         HumanMessage(content=question),
#         AIMessage(content=ai_msg_1["answer"]),
#     ]
# )

# print(ai_msg_1['answer'])

In [54]:
chat_history

[]

In [58]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [59]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [60]:
conversational_rag_chain.invoke(
    {"input": "Which Bank did Tracy go to?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


"I don't know. The context only mentions a customer named John Perez, not Tracy, and it's about ordering pizza from Pizza Loco, not going to a bank."